In [8]:
import db_operations as dbop
from constants import TOKEN, STOCK_DAY, INDEX_DAY,TABLE,COLUMNS
import pandas as pd
db_type = "sqlite3"
conn = dbop.connect_db(db_type)
cursor = conn.cursor()

# Get all trading dates from stock index table.
cursor.execute("select distinct date from {0} order by date".format(
                    INDEX_DAY[TABLE]))
dates = pd.DataFrame(cursor.fetchall(), columns=["date"])["date"]
dates = sorted(dates.unique())
print(len(dates))

# Get day data of all stocks from stocks table.
cursor.execute("select * from {0}".format(STOCK_DAY[TABLE]))
df_stocks_day = pd.DataFrame(cursor.fetchall())
df_stocks_day.columns = dbop.cols_from_cur(cursor)
df_stocks_day = df_stocks_day.set_index("date")
print(df_stocks_day.shape)

2149


(123663, 8)


In [ ]:
for code, df_single_stock in df_stocks_day.groupby(by="code"):
    df_single_stock = df_single_stock.sort_index()
    for i in range(1,len(dates)):
        if dates[i] in df_single_stock:
            is_changed = False
            # Note that by using groupby code, code is always not null.
            if df_single_stock.loc[dates[i]].isna().all()[1:].all():
                # All columns except code is null.
                df_single_stock.loc[dates[i],["open","high","low","close","adj_factor"]] = df_single_stock.loc[dates[i-1],["open","high","low","close","adj_factor"]]
                df_single_stock.loc[dates[i],["vol","amt"]] = [0,0]
                is_changed = True
            elif df_single_stock.loc[dates[i]].isna().all()[1:-1].all():
                # All columns excpet code and adj_factor is null.
                df_single_stock.loc[dates[i],["open","high","low","close"]] = df_single_stock.loc[dates[i-1],["open","high","low","close"]]
                df_single_stock.loc[dates[i],["vol","amt"]] = [0,0]
            elif df_single_stock.loc[dates[i]].isna().all()[1:-3].all():
                if (df_single_stock.loc[dates[i],["vol","amt"]] != [0,0]).any():
                    raise ValueError("Error row: {}".format(df_single_stock.loc[dates[i]]))
            
        
        